In [48]:
from glob import iglob
import pandas as pd

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from fuzzymatcher import link_table, fuzzy_left_join

path = r'./../airflow/raw/olx_anuncios/*.csv'

all_rec = iglob(path, recursive=True)
dataframes = (pd.read_csv(f, encoding='latin') for f in all_rec)
big_dataframe = pd.concat(dataframes, ignore_index=True)


In [49]:
df_olx_veiculos = big_dataframe.copy()
df_olx_veiculos = df_olx_veiculos.drop(labels=['id', 'index', 'car_features', 'cartype', 'category', 'doors',
                                               'end_tag', 'exchange', 'owner', 'motorpower', 'gearbox', 'car_steering', 'preco_anterior', 'financial', 'carcolor',
                                               'fuel'], axis=1)
df_olx_veiculos = df_olx_veiculos.rename(columns={'mileage': 'KM', 'regdate': 'Ano_Fabricacao',
                                                  'vehicle_brand': 'Marca', 'vehicle_model': 'Modelo', 'preco_anuncio': 'Preco', 'anunciante': 'vendedor'})
df_olx_veiculos['Preco'] = df_olx_veiculos['Preco'].str.replace('R', '')
df_olx_veiculos['Preco'] = df_olx_veiculos['Preco'].str.replace('$', '')
df_olx_veiculos['Preco'] = df_olx_veiculos['Preco'].str.replace('.', '')
df_olx_veiculos['Preco'] = df_olx_veiculos['Preco'].str.replace(',00', '')
df_olx_veiculos['Preco'] = pd.to_numeric(df_olx_veiculos['Preco'], errors='coerce')
df_olx_veiculos


C:\Users\mayco\AppData\Local\Temp/ipykernel_20220/2407417250.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_olx_veiculos['Preco'] = df_olx_veiculos['Preco'].str.replace('$', '')
C:\Users\mayco\AppData\Local\Temp/ipykernel_20220/2407417250.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_olx_veiculos['Preco'] = df_olx_veiculos['Preco'].str.replace('.', '')


,KM,Ano_Fabricacao,Marca,Modelo,anuncio_id_olx,link_anuncio_olx,Preco,vendedor,bairro,cidade,uf,regiao
0,100000,2013,FIAT,FIAT SIENA EL 1.4 MPI FIRE FLEX 8V 4P,568389900,https://ms.olx.com.br/mato-grosso-do-sul/autos...,32990.0,genilson,Centro,Dourados,MS,"Mato Grosso do Sul, MS"
1,31000,2019,RENAULT,RENAULT SANDERO STEPWAY EXP. FLEX 1.6 16V 5P,674147512,https://ms.olx.com.br/mato-grosso-do-sul/autos...,45990.0,Jose Augusto,Itanhangá Park,Campo Grande,MS,"Mato Grosso do Sul, MS"
2,0,2017,TOYOTA,TOYOTA COROLLA DYNAMIC 2.0 FLEX 16V AUT.,704398324,https://ms.olx.com.br/mato-grosso-do-sul/autos...,NaN,LEANDRO VEICULOS,Vila Bandeirante,Campo Grande,MS,"Mato Grosso do Sul, MS"
3,133300,1995,BMW,BMW 318TI COMPACT MEC,741266471,https://ms.olx.com.br/mato-grosso-do-sul/autos...,29990.0,negociosveiculos,Vila Rica,Campo Grande,MS,"Mato Grosso do Sul, MS"
4,200000,2001,HONDA,HONDA CIVIC SEDAN LX 1.7 16V 115CV MEC. 4P,771571406,https://ms.olx.com.br/mato-grosso-do-sul/autos...,16990.0,Bee SemiNovos,Vila Carvalho,Campo Grande,MS,"Mato Grosso do Sul, MS"
...,...,...,...,...,...,...,...,...,...,...,...,...
5308,125000,2014,FORD,FORD FUSION 2.5L I-VCT FLEX AUT.,964706475,https://ms.olx.com.br/mato-grosso-do-sul/autos...,66000.0,RHIANE,Jardim Canguru,Campo Grande,MS,"Mato Grosso do Sul, MS"
5309,78000,2014,FIAT,FIAT PALIO 1.0 ECONOMY FIRE FLEX 8V 2P,964707537,https://ms.olx.com.br/mato-grosso-do-sul/autos...,19900.0,Mateus,Vila Almeida,Campo Grande,MS,"Mato Grosso do Sul, MS"
5310,87000,2012,VW - VOLKSWAGEN,VW - VOLKSWAGEN SPACEFOX SPORTLINE/HIGHLINE 1...,964709033,https://ms.olx.com.br/mato-grosso-do-sul/autos...,380009.0,fabio ramos,Jardim Leblon,Campo Grande,MS,"Mato Grosso do Sul, MS"
5311,126000,2008,VW - VOLKSWAGEN,VW - VOLKSWAGEN GOL 1.0 TREND/ POWER 8V 4P,964712145,https://ms.olx.com.br/mato-grosso-do-sul/autos...,26000.0,Evelyn S.,Centro,Campo Grande,MS,"Mato Grosso do Sul, MS"


In [51]:
df_marca_modelo = pd.read_csv(
    '../airflow/raw/scrapping_marcas_modelos_fipe.csv', encoding='latin').drop('Unnamed: 0', axis=1)
df_fuzzy = fuzzy_left_join(df_olx_veiculos, df_marca_modelo, 'Modelo', 'nome_modelo')
df_fuzzy = df_fuzzy.drop(labels=['__id_left', '__id_right'], axis=1)
df_fuzzy.sort_values(by='best_match_score')
df_fuzzy.to_csv(f'../airflow/staging/anuncios_olx_fuzzy.csv',
                mode='a', header=False, encoding='latin')
df_fuzzy.head(10)


,best_match_score,KM,Ano_Fabricacao,Marca,Modelo,anuncio_id_olx,link_anuncio_olx,Preco,vendedor,bairro,cidade,uf,regiao,nome_marca,id_marca,nome_modelo,id_modelo
0,1.077383,100000,2013,FIAT,FIAT SIENA EL 1.4 MPI FIRE FLEX 8V 4P,568389900,https://ms.olx.com.br/mato-grosso-do-sul/autos...,32990.0,genilson,Centro,Dourados,MS,"Mato Grosso do Sul, MS",Fiat,21.0,Siena EL 1.4 mpi Fire Flex 8V 4p,5361.0
2,0.953587,31000,2019,RENAULT,RENAULT SANDERO STEPWAY EXP. FLEX 1.6 16V 5P,674147512,https://ms.olx.com.br/mato-grosso-do-sul/autos...,45990.0,Jose Augusto,Itanhangá Park,Campo Grande,MS,"Mato Grosso do Sul, MS",Renault,48.0,SANDERO STEPWAY EXP. Flex 1.6 16V 5p,8065.0
3,0.685940,0,2017,TOYOTA,TOYOTA COROLLA DYNAMIC 2.0 FLEX 16V AUT.,704398324,https://ms.olx.com.br/mato-grosso-do-sul/autos...,NaN,LEANDRO VEICULOS,Vila Bandeirante,Campo Grande,MS,"Mato Grosso do Sul, MS",Toyota,56.0,Corolla Dynamic 2.0 Flex 16V Aut.,7746.0
4,0.476640,133300,1995,BMW,BMW 318TI COMPACT MEC,741266471,https://ms.olx.com.br/mato-grosso-do-sul/autos...,29990.0,negociosveiculos,Vila Rica,Campo Grande,MS,"Mato Grosso do Sul, MS",BMW,7.0,318Ti Compact Mec,159.0
5,1.040626,200000,2001,HONDA,HONDA CIVIC SEDAN LX 1.7 16V 115CV MEC. 4P,771571406,https://ms.olx.com.br/mato-grosso-do-sul/autos...,16990.0,Bee SemiNovos,Vila Carvalho,Campo Grande,MS,"Mato Grosso do Sul, MS",Honda,25.0,Civic Sedan LX 1.7 16V 115cv Mec. 4p,1264.0
6,0.499395,83000,2013,TOYOTA,TOYOTA COROLLA ALTIS 2.0 FLEX 16V AUT.,775565211,https://ms.olx.com.br/mato-grosso-do-sul/autos...,62000.0,patrick,Jardim Jacy,Campo Grande,MS,"Mato Grosso do Sul, MS",Toyota,56.0,Corolla ALTIS/A.Premiu. 2.0 Flex 16V Aut,9300.0
7,NaN,0,2014,NaN,,819357935,https://ms.olx.com.br/mato-grosso-do-sul/autos...,NaN,STRADA AUTO PEÇAS,Vila Piratininga,Campo Grande,MS,"Mato Grosso do Sul, MS",NaN,NaN,NaN,NaN
8,0.857679,12600,1991,LADA,LADA NIVA 1.6 RC/ PANTANAL 4X4,819771805,https://ms.olx.com.br/mato-grosso-do-sul/autos...,25800.0,wilson mena,Vila Bandeirante,Campo Grande,MS,"Mato Grosso do Sul, MS",Lada,32.0,Niva 1.6 RC/ Pantanal 4x4,1439.0
9,1.095050,95000,2015,VW - VOLKSWAGEN,VW - VOLKSWAGEN AMAROK HIGHLINE CD 2.0 16V TDI...,822086043,https://ms.olx.com.br/mato-grosso-do-sul/autos...,110000.0,ALEMÃO - 99912-1150,Itanhangá Park,Campo Grande,MS,"Mato Grosso do Sul, MS",VW - VolksWagen,59.0,AMAROK Highline CD 2.0 16V TDI 4x4 Dies.,5261.0
10,0.686178,140000,2012,FORD,FORD FIESTA 1.0 8V FLEX/CLASS 1.0 8V FLEX 5P,824518082,https://ms.olx.com.br/mato-grosso-do-sul/autos...,22990.0,Bee SemiNovos,Vila Carvalho,Campo Grande,MS,"Mato Grosso do Sul, MS",Ford,22.0,Fiesta 1.0 8V Flex/Class 1.0 8V Flex 5p,4135.0
